In [1]:
import os
import torch
import torchvision
from torch import nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchinfo
import modules

In [2]:
num_workers = os.cpu_count()
print(num_workers)
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

10
mps


In [7]:
train_dir = "modules/data/train_organized/"
test_dir = "modules/data/test_organized/"


In [4]:
weights = torchvision.models.EfficientNet_V2_L_Weights.DEFAULT
weights

EfficientNet_V2_L_Weights.IMAGENET1K_V1

In [15]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.5, 0.5, 0.5]
    std=[0.5, 0.5, 0.5]
    interpolation=InterpolationMode.BICUBIC
)

In [10]:
from modules import data_setup
train_dataloader, test_dataloader = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir, transform=auto_transforms, batch_size=32, num_workers=num_workers)

In [14]:
model = torchvision.models.efficientnet_v2_l(weights=weights).to(device)
model

Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /Users/raghavmehta/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth


100%|██████████| 455M/455M [00:48<00:00, 9.74MB/s] 


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [17]:
from torchinfo import summary
summary(
    model=model,
    input_size=(32,3,480,480),
    col_names=["input_size","output_size","num_params","trainable"],
    col_width=20,
    row_settings=["var_names"]
)

KeyboardInterrupt: 

In [ ]:
loss_fn 

In [ ]:
from modules import engine

engine.train(
    model=model
)


/Users/raghavmehta/Downloads/Fruit-Freshness-Classification/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
